In [ ]:
import pandas as pd
import json
import os

In [ ]:
# open the jesonfile as dataframe
json_file = "C:\\Users\\user\\Desktop\\AI projects\\nlp_project_files\\kone_classification.json"
with open(json_file) as f:
    data = json.load(f)
    df_json=pd.DataFrame(data)

In [ ]:
print("The description of the dataset is: \n",df_json.info())
print("The number of labels in the dataset is: ",df_json['label'].nunique())
# count the rows for each language
df_json.groupby('culture').count()

In [ ]:
# check the maximum and minimum frequent for each label
df_json.groupby('label').count().sort_values(by=['text'], ascending=False)

In [ ]:
# choose the training source and drop the workflow
df_json_training= df_json.loc[df_json['source']== 'TRAINING',:]
df_json_training

In [ ]:
# choose the French culture
df_json_training_fr = df_json_training.loc[df_json_training['culture']=='fr-fr',:]

In [ ]:
# preprocessing step
import re

# some text cleaning functions
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc

def remove_special_char(text):
    special_char = r'[^\w\s]|.:,*"'
    remove_special_char = re.sub(pattern=special_char, repl=" ", string=text)
    return remove_special_char
df_json['text'] = df_json['text'].apply(lambda x: convert_to_lower(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_numbers(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_extra_white_spaces(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_special_char(x))

In [ ]:
df_json_training_fr['text'] = df_json_training_fr['text'].apply(lambda x: convert_to_lower(x))
df_json_training_fr['text'] = df_json_training_fr['text'].apply(lambda x: remove_numbers(x))
df_json_training_fr['text'] = df_json_training_fr['text'].apply(lambda x: remove_extra_white_spaces(x))
df_json_training_fr['text'] = df_json_training_fr['text'].apply(lambda x: remove_special_char(x))

In [ ]:
df_json_training_fr.duplicated().sum()

In [ ]:
df_clean = df_json_training_fr.drop_duplicates()

In [ ]:
df_clean.duplicated().sum()

In [ ]:
# check the maximum and minimum frequent for each label
df_clean['label_id'] = df_clean['label'].factorize()[0]
# df_clean.groupby('label').count().sort_values(by=['text'], ascending=False)

In [ ]:
df_clean.groupby(['label', 'label_id']).count().sort_values(by=['text'], ascending=False)

In [ ]:
df_clean


In [ ]:
from numpy import mean
from numpy import std
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
 
# load the dataset
def load_dataset(df):
	# load the dataset as a numpy array
	data = df
	# retrieve numpy array
	data = df[['text', 'label']]
	# split into input and output elements
	X, y = data['text'], data['label']
	# label encode the target variable to have the classes 0 and 1
	y = LabelEncoder().fit_transform(y)
	return X, y

In [ ]:
## sentence embeding using sentence transformer
from sentence_transformers import SentenceTransformer

model_embed = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def embed(data, model_embed):
    sentences = data.values
    embeddings = model_embed.encode(sentences)
    return embeddings

In [ ]:
# # evaluate a model
# def evaluate_model(X, y, model):
# 	# define evaluation procedure
# 	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# 	# evaluate model
# 	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# 	return scores

In [ ]:
df = df_clean

In [ ]:
X, y = load_dataset(df)

In [ ]:
X= embed(X,model_embed)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20,
                                                    random_state = 30)

In [ ]:
model = RandomForestClassifier(n_estimators=1000, class_weight= 'balanced')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
# evaluate the model

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
	# evaluate model
scores = cross_val_score(model,X_train,y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
from sklearn.metrics import classification_report
model = RandomForestClassifier(n_estimators=1000, class_weight= 'balanced')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

# evaluate the model

print(classification_report(y_test, y_pred))


# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# 	# evaluate model
# scores = cross_val_score(model,X_train,y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# # summarize performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)
ax.xaxis.set_ticklabels(y_test)
ax.yaxis.set_ticklabels(y_test)
_ = ax.set_title(
    f"Confusion Matrix for {model.__class__.__name__}\non the original documents")

In [ ]:
model = RandomForestClassifier(n_estimators=1000, class_weight='balanced')
# evaluate the model
scores = evaluate_model(X, y, model)
# summarize performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

training the data without all classes with one sample

In [ ]:
minority_classes = ['OOS','LD01','SH02','SH26','LD16','CAR25','CAR19','LD11','DOC10','MR15','CAR15','SH03','DOC08','CAR16','SH14','LD14','CD02','DOC09','SH27','CW01']

In [ ]:
df_new = df_clean[df_clean.label.isin(minority_classes) ==False]

In [ ]:
list(df_new.label_id.unique())

In [ ]:
df_new.groupby(['label', 'label_id']).count().sort_values(by=['text'], ascending=False)

In [ ]:
X, y = load_dataset(df_new)

In [ ]:
X= embed(X,model_embed)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20,
                                                    random_state = 0)

In [ ]:
labels_in_df_new = list(df_new.label_id.values)

In [ ]:
weights = {}

values = [1.0 ,0, 2.0]
for i in labels_in_df_new:
    if i == 30 or i == 31 or i== 33 or i == 34 or i == 39:
       continue
    elif i == 4 or i == 6 or i == 7 or i == 9:
        a= {i:values[0]}
        weights.update(a)
    else:
        a= {i:values[-1]}
        weights.update(a)
    

In [ ]:
weights

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
import seaborn as sns

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(40,20))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=df_new.label.values, 
            yticklabels=df_new.label.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')